# Noise reduction + wavelet transform

**In this notebook, I decompose the signals using a Wavelet Transform (WT) from which entropy measures are computed and stored in a dataframe**

Inspired from a Kaggle notebook -> https://www.kaggle.com/kiran3799/epilepsy-prediction-notebook

and from a paper -> https://journals.sagepub.com/doi/10.1177/1550147720911009

## Overall process

From each segments:

* Noise reduction using ICA algorithm (fastICA from sklearn)
* Wavelet transform
* Features extraction (mean absolute value, average power, standard deviation, variance, mean, skewness, and Shannon entropy)
* Classification (KNN, SVM, ...)

## Imports

In [1]:
import pywt
import os
import random
import numpy as np
from pyentrp import entropy
from scipy.stats import median_absolute_deviation
from sklearn.preprocessing import RobustScaler
import pandas as pd
import scipy.io
from scipy.signal import spectrogram
from scipy.signal import resample
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Load test data

In [2]:
interictal_test = pd.read_csv(os.path.join(os.getcwd(),'../raw_data/Patient_1_csv/train_segments_unlabelled/interictal_segment_50.csv'))
preictal_test = pd.read_csv(os.path.join(os.getcwd(),'../raw_data/Patient_1_csv/train_segments_unlabelled/preictal_segment_1.csv'))

In [3]:
interictal_test.head()

,0,1,2,3,4,5,6,7,8,9,...,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
0,11.205265,167.071511,75.419953,162.286907,211.937037,153.678963,177.927322,166.478143,192.856010,221.013682,...,-44.464473,-71.423095,-80.289187,-107.851673,-169.742237,-208.624011,-171.272735,-66.927319,-72.434430,-58.139543
1,-132.270116,-55.462781,-170.133062,-41.973859,-38.518123,-146.716251,-98.169147,-119.221746,-49.319147,71.344001,...,511.624613,505.597751,522.420786,417.045427,341.989907,329.557642,315.109625,417.620291,333.218627,231.127537
2,-38.381349,182.387193,102.018862,133.921966,160.131290,134.429295,156.295005,137.161324,151.880594,163.515580,...,-291.956694,-274.804727,-285.771946,-328.505936,-366.997749,-356.998013,-319.989103,-272.702536,-297.534086,-280.593203
3,254.676796,483.462039,369.077160,382.093638,414.483458,377.479944,392.027705,397.471578,427.565397,423.449111,...,-798.001707,-774.993283,-829.218513,-906.712882,-932.956457,-906.870225,-849.384170,-792.310151,-823.089521,-813.866863
4,259.512509,475.155292,297.151224,311.565135,372.448171,324.682749,330.145081,357.385271,394.372344,374.583463,...,-108.598073,-63.119489,-160.522970,-250.015801,-238.096238,-183.450695,-125.356757,-100.697214,-158.033309,-170.755953


## ICA noise reduction

In [4]:
from sklearn.decomposition import FastICA

In [5]:
pwd

'/Users/commander/code/jhupiterz/SafeSeizure/notebooks'

In [6]:
ica_transformer = FastICA(random_state=0)
interictal_transformed = ica_transformer.fit_transform(interictal_test)
preictal_transformed = ica_transformer.fit_transform(preictal_test)

In [7]:
interictal_transformed

array([[ 2.79559186e-01, -2.72092172e-01, -2.93809897e-01,
        -3.12203227e-01,  2.65401851e-01, -2.96260961e-01,
         2.79149444e-01, -2.11194814e-01, -2.28483027e-01,
         7.10395767e-02,  2.68024737e-01,  2.76339180e-01,
         2.84082435e-01,  3.00091901e-01, -2.90982277e-02],
       [ 1.79062512e-01,  2.92113101e-01, -3.08715934e-01,
         2.91760481e-01,  1.52991476e-01,  1.52342329e-01,
        -2.54847394e-01,  3.23011096e-01,  3.04796664e-01,
         8.26555669e-02,  3.61381670e-01, -2.21572231e-01,
         2.61350004e-01,  2.86239960e-01, -3.27848201e-02],
       [ 2.97732693e-01, -3.09317140e-01,  3.15892124e-01,
         2.30696925e-01, -2.98271380e-01,  3.18041634e-01,
        -2.66686402e-01, -1.40942337e-01, -2.26789160e-01,
         6.93938079e-02,  3.24155613e-01,  2.50142453e-01,
        -2.90652672e-01,  2.81857052e-01, -2.80064766e-02],
       [ 3.42711023e-01,  3.58339856e-01,  3.24146259e-01,
        -3.48856171e-01,  3.13375124e-01, -3.30743183

## Discrete Wavelet Transform

In [9]:
wavelet = pywt.Wavelet('db4')
coeffs = pywt.wavedec(interictal_transformed, wavelet, level = 1)

In [10]:
coeffs

[array([[ 4.31746485e-03, -5.28466342e-01,  3.78331313e-01,
         -3.54081405e-01, -1.82127515e-01,  8.21918306e-02,
         -2.68241535e-01,  1.14977090e-01,  4.41513852e-01,
          2.21303084e-01,  1.15206685e-01],
        [ 2.90846528e-01, -1.19168559e-02,  3.70246580e-01,
          1.14210625e-02,  2.96473086e-01, -6.58971212e-02,
          2.93305557e-01,  3.22709467e-01,  9.34983619e-02,
          1.88468916e-01,  1.94355858e-01],
        [ 7.68577302e-02,  1.82044866e-02,  2.72916789e-01,
          9.86313513e-02,  7.79536410e-02,  4.68986607e-02,
         -3.72667729e-01,  2.57293236e-01,  7.89600318e-02,
          6.07486186e-02,  1.27376660e-01],
        [-1.30318689e-01,  3.29134176e-01,  4.76145341e-01,
          4.66746817e-01,  1.11773721e-01, -4.47511283e-01,
          4.18536583e-01, -9.09721259e-02, -5.04091971e-02,
         -2.94167473e-01, -3.40091534e-01],
        [-3.81973750e-01,  2.57813767e-01, -2.99952372e-01,
          2.90895047e-01, -3.99616509e-01,  

## Feature extraction

In [12]:
# mean absolute value, average power, standard deviation, variance, mean, skewness, and Shannon entropy

In [13]:
def flatten_list(l):
    flat_list = []
    for sublist in l:
        for item in sublist:
            flat_list.append(item)
    return flat_list

In [14]:
def mean_absolute_value(array):
    absolute_values = []
    for i in range(len(array)):
        absolute_values.append(abs(array[i]))
    mean_absolute_value = np.asarray(absolute_values).sum()*(1/len(array))
    #print(mean_absolute_value)
    return mean_absolute_value

In [15]:
def average_power(array):
    average_power_values = []
    for i in range(len(array)):
        average_power_values.append(abs(array[i])**2)
    average_power_value = np.asarray(average_power_values).sum()*(1/len(array))
    #print(average_power_value)
    return average_power_value

In [16]:
def shan(d1):
    sh1=[]
    d1=np.rint(d1)
    for i in range(d1.shape[0]):
        X=d1[i]
        sh1.append(entropy.shannon_entropy(X))
    return(sh1)

In [17]:
def feature_extraction(decomposed_signals):
    mav = []
    avp = []
    std = []
    var = []
    mean = []
    for coeff in decomposed_signals:
        mav_electrode = []
        avp_electrode = []
        std_electrode = []
        var_electrode = []
        mean_electrode = []
        for electrode in coeff:
            mav_electrode.append(mean_absolute_value(electrode))
            avp_electrode.append(average_power(electrode))
            std_electrode.append(np.std(electrode))
            var_electrode.append(np.var(electrode))
            mean_electrode.append(np.mean(electrode))
        mav.append(mav_electrode)
        avp.append(avp_electrode)
        std.append(std_electrode)
        var.append(var_electrode)
        mean.append(mean_electrode)
    mav = flatten_list(mav)
    avp = flatten_list(avp)
    std = flatten_list(std)
    var = flatten_list(var)
    mean = flatten_list(mean)
    shan_ent = []
    for i in range(len(decomposed_signals)):
        shan_ent.append(shan(coeffs[i]))
    shan_ent = flatten_list(shan_ent)
    data = np.vstack((np.array(mav), np.array(avp), 
                     np.array(std), np.array(var), np.array(mean), np.array(shan_ent)))
    #data = pd.DataFrame(data)
    features = data.T
    #features.columns = ['mean_abs_value', 'average_power', 'std', 'var', 'mean', 'shannon_entropy']

    return features

In [18]:
feature_extraction(coeffs)

array([[ 0.24461437,  0.08455317,  0.29077144,  0.08454803,  0.00226587,
         0.43949699],
       [ 0.19446722,  0.05330833,  0.14419885,  0.02079331,  0.18031922,
        -0.        ],
       [ 0.13531899,  0.02999613,  0.15947291,  0.02543161,  0.06756123,
        -0.        ],
       [ 0.28689154,  0.10643869,  0.32368744,  0.10477356,  0.04080603,
        -0.        ],
       [ 0.26161401,  0.08331582,  0.2886057 ,  0.08329325, -0.0047503 ,
        -0.        ],
       [ 0.26156309,  0.08900265,  0.29623972,  0.08775797, -0.03528006,
        -0.        ],
       [ 0.15215356,  0.04061983,  0.1726581 ,  0.02981082, -0.1039664 ,
        -0.        ],
       [ 0.28739981,  0.10417415,  0.26877373,  0.07223932,  0.17870319,
         0.43949699],
       [ 0.23745887,  0.07706929,  0.27284162,  0.07444255, -0.05125169,
         0.43949699],
       [ 0.21929178,  0.07328429,  0.24945685,  0.06222872, -0.10514546,
        -0.        ],
       [ 0.18959654,  0.05524772,  0.22136458,  0.

## Feature extraction for each segment

In [20]:
# Builds list of files fro Dog_1
folder_path = '/Users/commander/code/jhupiterz/SafeSeizure/raw_data/Dog_1_csv'
files_Dog1 = []

for f in os.listdir(folder_path):
    if f.startswith('preictal') | f.startswith('interictal'):
        files_Dog1.append(f)
len(files_Dog1)

504

In [21]:
# Pipeline: ICA -> Wavelet -> Feature extraction as array -> build target vector
ica_transformer = FastICA(n_components= 10,max_iter= 3000000, random_state=0, tol=0.1)
wavelet = pywt.Wavelet('db4')
features = []
target = []
for file in files_Dog1:
    segment = pd.read_csv(os.path.join(folder_path, file))
    #segment_scaled = robust_scaler.fit_transform(segment)
    segment_transformed = ica_transformer.fit_transform(segment)
    coeffs = pywt.wavedec(segment_transformed, wavelet, level = 1)
    segment_features = feature_extraction(coeffs)
    np.asarray(features.append(segment_features))
    if file.startswith('interictal'):
        target.append(0)
    elif file.startswith('preictal'):
        target.append(1)
    print(file)
print(len(features), (features[0].shape), target, len(target))

interictal_segment_349.csv
interictal_segment_361.csv
interictal_segment_407.csv
interictal_segment_413.csv
interictal_segment_375.csv
interictal_segment_188.csv
interictal_segment_163.csv
interictal_segment_177.csv
interictal_segment_50.csv
interictal_segment_44.csv
interictal_segment_78.csv
interictal_segment_93.csv
interictal_segment_87.csv
interictal_segment_229.csv
interictal_segment_215.csv
interictal_segment_201.csv
interictal_segment_200.csv
interictal_segment_214.csv
interictal_segment_228.csv
interictal_segment_86.csv
interictal_segment_92.csv
interictal_segment_79.csv
interictal_segment_45.csv
interictal_segment_51.csv
interictal_segment_176.csv
interictal_segment_162.csv
interictal_segment_189.csv
interictal_segment_412.csv
interictal_segment_374.csv
interictal_segment_360.csv
interictal_segment_406.csv
interictal_segment_348.csv
interictal_segment_389.csv
interictal_segment_438.csv
interictal_segment_376.csv
interictal_segment_410.csv
interictal_segment_404.csv
interictal_

interictal_segment_443.csv
interictal_segment_325.csv
interictal_segment_319.csv
interictal_segment_2.csv
interictal_segment_133.csv
interictal_segment_127.csv
interictal_segment_28.csv
interictal_segment_14.csv
interictal_segment_286.csv
interictal_segment_292.csv
interictal_segment_245.csv
interictal_segment_251.csv
interictal_segment_279.csv
interictal_segment_278.csv
interictal_segment_250.csv
interictal_segment_244.csv
interictal_segment_293.csv
interictal_segment_287.csv
interictal_segment_15.csv
interictal_segment_29.csv
interictal_segment_126.csv
interictal_segment_132.csv
interictal_segment_3.csv
interictal_segment_318.csv
interictal_segment_442.csv
interictal_segment_324.csv
interictal_segment_330.csv
preictal_segment_10.csv
interictal_segment_456.csv
interictal_segment_326.csv
interictal_segment_440.csv
interictal_segment_454.csv
preictal_segment_12.csv
interictal_segment_332.csv
interictal_segment_468.csv
interictal_segment_1.csv
interictal_segment_124.csv
interictal_segmen

In [22]:
features_dog1 = np.asarray(features)
features_dog1.shape

(504, 32, 6)

In [23]:
# Builds list of files for Dog_2
folder_path = '/Users/commander/code/jhupiterz/SafeSeizure/raw_data/Dog_2_csv/'
files_Dog2 = []

for f in os.listdir(folder_path):
    if f.startswith('preictal') | f.startswith('interictal'):
        files_Dog2.append(f)
len(files_Dog2)

542

In [24]:
# Pipeline: ICA -> Wavelet -> Feature extraction as array -> build target vector
features_2 = []
target_2 = []
for file in files_Dog2:
    segment = pd.read_csv(os.path.join(folder_path, file))
    #segment_scaled = robust_scaler.fit_transform(segment)
    segment_transformed = ica_transformer.fit_transform(segment)
    coeffs = pywt.wavedec(segment_transformed, wavelet, level = 1)
    segment_features = feature_extraction(coeffs)
    np.asarray(features_2.append(segment_features))
    if file.startswith('interictal'):
        target_2.append(0)
    elif file.startswith('preictal'):
        target_2.append(1)
    print(file)
print(len(features_2), (features_2[0].shape), target_2, len(target_2))

interictal_segment_349.csv
interictal_segment_361.csv
interictal_segment_407.csv
preictal_segment_41.csv
interictal_segment_413.csv
interictal_segment_375.csv
interictal_segment_188.csv
interictal_segment_163.csv
interictal_segment_177.csv
interictal_segment_50.csv
interictal_segment_44.csv
interictal_segment_78.csv
interictal_segment_93.csv
interictal_segment_87.csv
interictal_segment_229.csv
interictal_segment_215.csv
interictal_segment_201.csv
interictal_segment_200.csv
interictal_segment_214.csv
interictal_segment_228.csv
interictal_segment_86.csv
interictal_segment_92.csv
interictal_segment_79.csv
interictal_segment_45.csv
interictal_segment_51.csv
interictal_segment_176.csv
interictal_segment_162.csv
interictal_segment_189.csv
interictal_segment_412.csv
interictal_segment_374.csv
interictal_segment_360.csv
preictal_segment_40.csv
interictal_segment_406.csv
interictal_segment_348.csv
interictal_segment_389.csv
interictal_segment_438.csv
interictal_segment_376.csv
interictal_segmen

interictal_segment_297.csv
interictal_segment_268.csv
interictal_segment_240.csv
interictal_segment_254.csv
interictal_segment_255.csv
interictal_segment_241.csv
interictal_segment_269.csv
interictal_segment_296.csv
interictal_segment_282.csv
interictal_segment_38.csv
interictal_segment_10.csv
interictal_segment_123.csv
interictal_segment_137.csv
interictal_segment_6.csv
interictal_segment_321.csv
interictal_segment_447.csv
preictal_segment_15.csv
interictal_segment_453.csv
interictal_segment_335.csv
preictal_segment_29.csv
interictal_segment_309.csv
interictal_segment_484.csv
interictal_segment_490.csv
interictal_segment_494.csv
interictal_segment_480.csv
interictal_segment_331.csv
interictal_segment_457.csv
preictal_segment_11.csv
interictal_segment_443.csv
interictal_segment_325.csv
preictal_segment_39.csv
interictal_segment_319.csv
interictal_segment_2.csv
interictal_segment_133.csv
interictal_segment_127.csv
interictal_segment_28.csv
interictal_segment_14.csv
interictal_segment_28

In [25]:
features_dog2 = np.asarray(features_2)
features_dog2.shape

(542, 32, 6)

In [26]:
# Stack all data in a ndarray
data = np.vstack((features_dog1, features_dog2))
len(data)

1046

In [27]:
# flatten the ndarray
flatten_data = data.reshape(len(data),-1)

In [28]:
# Concatenate targets into one column vector
target_dog1 = pd.Series(np.array(target))
target_dog2 = pd.Series(np.array(target_2))
targets = pd.concat([target_dog1, target_dog2], axis = 0, ignore_index = True)

## Logistic regression

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import SMOTE

log_reg = LogisticRegression()
sm = SMOTE(random_state=42)
#X_res, y_res = sm.fit_resample(X,y)

X = flatten_data
y = targets
X_resampled, y_resampled = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

cv_results = cross_validate(log_reg, X_resampled, y_resampled, cv=5, scoring='accuracy')
cv_results

{'fit_time': array([0.04441214, 0.04013491, 0.03806591, 0.03897619, 0.03698421]),
 'score_time': array([0.00070477, 0.00051999, 0.00052094, 0.00049376, 0.00043869]),
 'test_score': array([0.78061224, 0.7627551 , 0.77295918, 0.74489796, 0.72193878])}

In [34]:
log_reg.fit(X_train, y_train)

LogisticRegression()

In [35]:
log_reg.score(X_test,y_test)

0.8095238095238095

In [36]:
log_reg.predict(X_test)

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,

## Support Vector Machines

### Linear kernel

In [38]:
from sklearn.svm import SVC

svc_linear = SVC(kernel='linear')
cv_results = cross_validate(svc_linear, X_resampled, y_resampled, cv=5, scoring='accuracy')
cv_results

{'fit_time': array([0.12440491, 0.10278177, 0.08672523, 0.08094287, 0.10102582]),
 'score_time': array([0.01951313, 0.01497006, 0.01378798, 0.01609802, 0.01695395]),
 'test_score': array([0.81632653, 0.80102041, 0.83163265, 0.7627551 , 0.7755102 ])}

In [39]:
svc_linear.fit(X_train, y_train)

SVC(kernel='linear')

In [40]:
svc_linear.score(X_test,y_test)

0.8299319727891157

### Polynomial kernel

In [52]:
svc_poly = SVC(kernel='poly')
cv_results = cross_validate(svc_poly, X_resampled, y_resampled, cv=5, scoring='accuracy')
cv_results

{'fit_time': array([0.09469509, 0.07349777, 0.06956077, 0.06941009, 0.0644238 ]),
 'score_time': array([0.0107758 , 0.01140618, 0.01095128, 0.01055598, 0.01041532]),
 'test_score': array([0.96428571, 0.95663265, 0.95153061, 0.94132653, 0.94642857])}

In [53]:
svc_poly.fit(X_train, y_train)
svc_poly.score(X_test,y_test)

0.9574829931972789

## KNeighbors Classifier

In [43]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

cv_results = cross_validate(knn, X_resampled, y_resampled, cv=5, scoring='accuracy')
cv_results

{'fit_time': array([0.00390506, 0.00125694, 0.00143623, 0.00130105, 0.00130796]),
 'score_time': array([0.02964711, 0.02317023, 0.02020001, 0.01874995, 0.01831603]),
 'test_score': array([0.75510204, 0.77040816, 0.76020408, 0.71938776, 0.69642857])}

In [44]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [45]:
knn.score(X_test,y_test)

0.7653061224489796

## Ensemble classifiers

In [46]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=10)

cv_results = cross_validate(random_forest, X_resampled, y_resampled, cv=5, scoring='accuracy')
cv_results

{'fit_time': array([0.10651803, 0.10415101, 0.08810806, 0.08871388, 0.08302402]),
 'score_time': array([0.00284004, 0.00175595, 0.00169706, 0.00157118, 0.00269008]),
 'test_score': array([0.93367347, 0.99234694, 0.9872449 , 0.96938776, 0.97193878])}

In [48]:
random_forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10)

In [49]:
random_forest.score(X_test,y_test)

0.9591836734693877